In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import meterdatalogic as ml

In [13]:
df = ml.ingest.from_nem12("./data/QB06066551_20250101_20251031_20251107130416_ENERGEXP_DETAILED.csv", tz="Australia/Brisbane")
ml.validate.assert_canon(df)
bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]
df

,nmi,channel,flow,kwh,cadence_min
t_start,,,,,
2025-01-01 00:00:00+10:00,QB06066551,B1,grid_export_solar,0.0000,5
2025-01-01 00:00:00+10:00,QB06066551,E1,grid_import,0.0257,5
2025-01-01 00:05:00+10:00,QB06066551,E1,grid_import,0.0174,5
2025-01-01 00:05:00+10:00,QB06066551,B1,grid_export_solar,0.0000,5
2025-01-01 00:10:00+10:00,QB06066551,B1,grid_export_solar,0.0000,5
...,...,...,...,...,...
2025-10-30 23:45:00+10:00,QB06066551,B1,grid_export_solar,0.0000,5
2025-10-30 23:50:00+10:00,QB06066551,E1,grid_import,0.0469,5
2025-10-30 23:50:00+10:00,QB06066551,B1,grid_export_solar,0.0000,5


In [14]:
monthly = ml.transform.groupby_month(df)  # columns: month + flows
monthly

flow,month,grid_export_solar,grid_import
0,2025-01,90.1570,609.0478
1,2025-02,91.0623,524.3936
2,2025-03,77.1941,439.5292
3,2025-04,75.2428,506.2042
4,2025-05,62.1714,415.0524
5,2025-06,48.8749,519.0934
6,2025-07,48.3685,762.1545
7,2025-08,71.9102,560.2731
8,2025-09,117.1583,418.6693
9,2025-10,97.5228,477.3780


In [15]:
## My current plan 
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand(
            name="all_times",     # must match the column name produced by transform.tou_bins
            start="00:00",
            end="24:00",          # full-day window
            rate_c_per_kwh=43.25  # $0.4325/kWh -> 43.25 cents
        )
    ],
    fixed_c_per_day=170.56,        # $1.7056/day -> 170.56 cents
    feed_in_c_per_kwh=60.0,        # $0.60/kWh -> 60.0 cents
    demand=None                    # no demand charge on this plan
)


In [16]:
cost = ml.pricing.estimate_monthly_cost(df, plan)  # month, components, total
cost

,month,energy_cost,demand_cost,fixed_cost,feed_in_credit,total
0,2025-01,263.413174,0.0,52.8736,-54.09420,262.192574
1,2025-02,226.800232,0.0,47.7568,-54.63738,219.919652
2,2025-03,190.096379,0.0,52.8736,-46.31646,196.653519
3,2025-04,218.933317,0.0,51.1680,-45.14568,224.955636
4,2025-05,179.510163,0.0,52.8736,-37.30284,195.080923
5,2025-06,224.507895,0.0,51.1680,-29.32494,246.350955
6,2025-07,329.631821,0.0,52.8736,-29.02110,353.484321
7,2025-08,242.318116,0.0,52.8736,-43.14612,252.045596
8,2025-09,181.074472,0.0,51.1680,-70.29498,161.947492
9,2025-10,206.465985,0.0,52.8736,-58.51368,200.825905


In [17]:
# Your retailer cycles (end is inclusive)
cycles = [
    ("2025-03-01", "2025-03-30"),
    ("2025-03-31", "2025-04-30"),
    ("2025-05-01", "2025-05-30"),
    ("2025-05-31", "2025-06-30"),
    ("2025-07-01", "2025-07-30"),
]

bills = ml.pricing.estimate_cycle_costs(
    df, plan, cycles,
    pay_on_time_discount=0.07,   
    include_gst=True            
)
# bills["total"] = bills["total"].round(2)
bills

,cycle,days_in_cycle,energy_cost,demand_cost,fixed_cost,feed_in_credit,pay_on_time_discount,gst,all_times,total
0,2025-03-01→2025-03-30,30,179.987989,0.0,51.1680,-44.98446,-16.18,21.50,416.1572,191.491529
1,2025-03-31→2025-04-30,31,229.041706,0.0,52.8736,-46.47768,-19.73,26.22,529.5762,241.927627
2,2025-05-01→2025-05-30,30,171.741339,0.0,51.1680,-36.82950,-15.60,20.73,397.0898,191.209839
3,2025-05-31→2025-06-30,31,232.276720,0.0,52.8736,-29.79828,-19.96,26.52,537.0560,261.912040
4,2025-07-01→2025-07-30,30,316.172638,0.0,51.1680,-28.52736,-25.71,34.16,731.0350,347.263278
